In [48]:
from detectron2.data import (
    DatasetCatalog,
    MetadataCatalog,
    build_detection_train_loader,
    build_detection_test_loader,
    DatasetMapper,
)
import argparse, os, json
from itertools import chain
from detectron2.utils.visualizer import Visualizer
from detectron2.data import detection_utils as utils
import tqdm

In [49]:
def get_data(args, file_name):
    def ret_data():
        output_path = os.path.join(args.dataroot, file_name)
        print(output_path)
        assert os.path.exists(output_path)
        with open(output_path, 'r') as fd:
            data = json.load(fd)
        fd.close()
        return data
    return ret_data

In [50]:
parser = argparse.ArgumentParser(description='Process some integers.')
parser.add_argument('--dataroot', type=str, \
                    default='/usr0/tma1/datasets/nuimages/detectron_data')
parser.add_argument('--train_file', type=str,\
                    default='v1.0-train_construction_detectron.json') 
parser.add_argument('--val_file', type=str,\
                    default='val_val_construction_detectron.json') 
parser.add_argument('--test_file', type=str,\
                    default='val_test_construction_detectron.json') 
args = parser.parse_args([])

In [51]:
categories = ['human.pedestrian.construction_worker', 'movable_object.barrier', 'movable_object.trafficcone', 'vehicle.construction']
category_dict = {i: name for i, name in enumerate(categories)}
data_train = DatasetCatalog.register("nuimages_train_trafficone", get_data(args, args.train_file))
MetadataCatalog.get("nuimages_train_trafficone").thing_classes = category_dict
data_val = DatasetCatalog.register("nuimages_val_trafficone", get_data(args, args.val_file))
MetadataCatalog.get("nuimages_val_trafficone").thing_classes = category_dict
data_test = DatasetCatalog.register("nuimages_test_trafficone", get_data(args, args.test_file))
MetadataCatalog.get("nuimages_test_trafficone").thing_classes = category_dict

AssertionError: Dataset 'nuimages_train_trafficone' is already registered!

In [52]:
output_dir = '/usr0/tma1/datasets/nuimages/object_detection_annotations/'
output_dir = os.path.join(output_dir, 'val_test_construction_detectron')
if not os.path.exists(output_dir):
    os.makedirs(output_dir)
visualize_dataset_name="nuimages_test_trafficone"
scale=5.0
show_image=False

In [53]:
def output(vis, fname):
    if show_image:
        print(fname)
        img = vis.get_image()[:, :, ::-1]
        # cv2.imshow("window", vis.get_image()[:, :, ::-1])
        figure(figsize=(10,10), dpi=80)
        plt.imshow(img)
        plt.show()
    else:
        filepath = os.path.join(output_dir, fname)
        # print("Saving to {} ...".format(filepath))
        vis.save(filepath)

In [ ]:
dicts = list(chain.from_iterable([DatasetCatalog.get(k) for k in [visualize_dataset_name]]))
metadata = MetadataCatalog.get(visualize_dataset_name)
for dic in tqdm.tqdm(dicts):
    img = utils.read_image(dic["file_name"], "RGB")
    visualizer = Visualizer(img, metadata=metadata, scale=scale)
    vis = visualizer.draw_dataset_dict(dic)
    output(vis, os.path.basename(dic["file_name"]))

/usr0/tma1/datasets/nuimages/detectron_data/val_test_construction_detectron.json


 54%|█████▍    | 1662/3061 [59:44<49:41,  2.13s/it]  